In [21]:
%pip install openai anthropic google-generativeai mistralai cohere

Note: you may need to restart the kernel to use updated packages.


In [22]:
import time
from openai import OpenAI
import os
import requests
from enum import Enum
import google.generativeai as genai
import anthropic
from mistralai import Mistral
import cohere
import pickle

TEMPERATURE = 0.7

In [17]:
cohere_client = cohere.ClientV2(api_key=os.environ['COHERE_API_KEY'])
openai_client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
perplexity_key = os.environ['PERPLEXITY_API_KEY']
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
anthropic_client = anthropic.Anthropic(api_key=os.environ['ANTHROPIC_API_KEY'])
mistral_client = Mistral(api_key=os.environ['MISTRAL_API_KEY'])
grok_client = OpenAI(api_key=os.environ['XAI_API_KEY'], base_url="https://api.x.ai/v1")
qwen_client = OpenAI(api_key=os.getenv("ALIBABA_API_KEY"),base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1")

KeyError: 'COHERE_API_KEY'

In [18]:
class LLM(Enum):
    CHATGPT = 1
    CLAUDE = 2
    PERPLEXITY = 3
    GEMINI = 4
    MISTRAL = 5
    COHERE = 6
    GROK = 7
    QWEN = 8

In [19]:
def ask(prompt, llm):
    if llm == LLM.CHATGPT:
        response = openai_client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="gpt-4o-mini",
            temperature=TEMPERATURE
        )
        return response.choices[0].message.content
    elif llm == LLM.PERPLEXITY:
        url = "https://api.perplexity.ai/chat/completions"

        payload = {
            "model": "llama-3.1-sonar-small-128k-online",
            "messages": [
                {
                    "role": "system",
                    "content": ""
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "max_tokens": 1000,
            "temperature": TEMPERATURE,
            "top_p": 0.9,
            "return_citations": True,
            "search_domain_filter": ["perplexity.ai"],
            "return_images": False,
            "return_related_questions": False,
            "search_recency_filter": "month",
            "top_k": 0,
            "stream": False,
            "presence_penalty": 0,
            "frequency_penalty": 1
        }
        headers = {
            "Authorization": f"Bearer {perplexity_key}",
            "Content-Type": "application/json"
        }

        try:
            response = requests.post(url, json=payload, headers=headers)
            response.raise_for_status()  # Raise an exception for bad status codes
            data = response.json()
            
            # Extract the actual response content from the Perplexity API response
            if 'choices' in data and len(data['choices']) > 0:
                return data['choices'][0]['message']['content']
            else:
                raise Exception("No content in response")
                
        except requests.exceptions.RequestException as e:
            raise Exception(f"Perplexity API error: {str(e)}")
    elif llm == LLM.GEMINI:
        model = genai.GenerativeModel(model_name="gemini-1.5-flash")
        response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=TEMPERATURE))
        return response.text
    elif llm == LLM.CLAUDE:
        message = anthropic_client.messages.create(
            model = 'claude-3-5-sonnet-20241022',
            max_tokens=1000,
            temperature=TEMPERATURE,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt,
                        }
                    ]
                }
            ]
        )
        return message.content[0].text
    elif llm == LLM.MISTRAL:
        response = mistral_client.chat.complete(
            model='mistral-large-latest',
            temperature=TEMPERATURE,
            messages = [
                {
                    "role": "user",
                    "content": prompt,
                }
            ]
        )
        return response.choices[0].message.content
    elif llm == LLM.COHERE:
        response = cohere_client.chat(
            model="command-r-plus-08-2024",
            temperature=TEMPERATURE,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        return response.message.content[0].text
    elif llm == LLM.GROK:
        response = grok_client.chat.completions.create(
            model="grok-beta",
            temperature=TEMPERATURE,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content
    elif llm == LLM.QWEN:       
        response = qwen_client.chat.completions.create(
            model="qwen-max-2025-01-25",
            messages=[
              {'role': 'system', 'content': 'You are a helpful assistant.'},
              {'role': 'user', 'content': 'Which number is larger, 9.11 or 9.8?'}
            ]
        )
        return response.choices[0].message